# Quick Map
  

In [1]:
import sys;sys.path.append('../html-maps')
from mapfun import *
from features import *
import pathlib as pl
import geopandas as gpd
from IPython.display import IFrame

### Read in data, projections, etc.

In [2]:
rootdir = pl.Path(r'..\sample_data')
crs = '+proj=aea +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs' 
name = 'New York'

gages = gpd.read_file(rootdir / 'sample_gages.geojson')
gages = quick_transform(gages, crs)

lines = gpd.read_file(rootdir / 'sample_lines.geojson')
lines = quick_transform(lines, crs)

polygons = gpd.read_file(rootdir / 'sample_polygons.geojson')
polygons = quick_transform(polygons, crs)

### Assign Map Center

In [3]:
xcoord, ycoord = get_centroid(gages)
m = interactive_map(ycoord, xcoord)

### For each shape added to the map, assign columnar data to show in pop-up

In [4]:
gages_info = {'Gage ID':'Gage_ID', 'Station':'Station_NM', 'URL':'NWISWeb'}
header= ['Company','Dewberry']
title = ''
params = column_styles(2,headers=header,size='55%',col_font='font-size: 10pt')
gages_descr = table_description(gages, params, gages_info, '<TH style="font-size:10pt;text-align:center">',
                                 title,change_header=True)
m = add_point_tables(gages, m, gages_descr, gages['Gage_ID'])

### Make popup for lines

In [5]:
line_info = {'REACH ID':'REACH_ID'}
header= ['Company','Dewberry']
title = ''
params = column_styles(2,headers=header,col_font='font-size: 10pt')
line_descr = table_description(lines, params,line_info, '<TH style="font-size:10pt;text-align:center">',
                                 title,change_header=True)
m = add_multiline(lines, m, line_descr, lines['REACH_ID'])

### Make popup for polygons

In [6]:
polygons

,OBJECTID,HUC12,NAME_1,PTS,Sqmi,Domains,Group_D,OBJECTID_1__,Label,Label_1,PM,Shape_Length,Shape_Area,geometry
0,5,020301030703,Lower Saddle River,STARRII,5.752770,2,1,5.0,Saddler River2,Saddler River2,John,29378.062107,1.489961e+07,(POLYGON ((-74.10029376904355 40.9516853143802...
1,6,020301030703,Lower Saddle River,STARRII,12.163356,1,1,6.0,Saddler River1,Saddler River1,John,45155.094771,3.150295e+07,(POLYGON ((-74.07272980569928 40.9682948790398...


In [7]:
poly_info = {'HUC12':'HUC12', 'Domain':'Domains', 'Label':'Label', 'PM':'PM'}
title = f'<caption style="font-size:14pt;text-align:center"><caption>{name}</caption><TH>'
header= ['Company','Dewberry']
params = column_styles(2,headers=header,size='60%',col_font='font-size: 10pt')
descriptions = table_description(polygons, params, poly_info, '<TH style="font-size:10pt;text-align:center">',
                                 title,change_header=False)
m = add_poly_table(polygons, m,descriptions, polygons['Label_1'])

### Add extra features to map

In [9]:
m = add_tiles(m,tile_type='World_Light_Gray_Base',services='/Canvas')
m = add_extra(m)

### Save to html

In [10]:
m.save(outfile='map.html')  
IFrame('map.html', width=800, height=650)

# END